In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Parameter
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
import torch.utils.data as data
from torch.nn import functional as F
#from torch_geometric.datasets import Planetoid
EPS = 1e-15

In [2]:
def computer_prediction(true_edges, recons_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    ap = average_precision_score(true_edges, predict_edges)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    return ap, auc

In [3]:
#####################################  测试用  #########################################
def predict_edges(lhs, rhs, anchor_emb):
    # power = float(alpha + 1) / 2
    power = 1.0
    
    # Calculate the attention score (in nodes) for each src node to each cluster
    src_norm_squared = torch.sum((lhs.unsqueeze(1) - anchor_emb)**2, 2)   # Distance from the node to the center of mass
    src_numerator = 1.0 / (1.0 + (src_norm_squared / DEC))
    src_numerator = src_numerator**power   
    src_soft_assignments = (src_numerator.t() / torch.sum(src_numerator, 1)).t()       #soft assignment using t-distribution
    
    # Calculate the attention score (in nodes) for each dst node to each cluster
    dst_norm_squared = torch.sum((rhs.unsqueeze(1) - anchor_emb)**2, 2)   # Distance from the node to the center of mass
    dst_numerator = 1.0 / (1.0 + (dst_norm_squared / DEC))
    dst_numerator = dst_numerator**power   
    dst_soft_assignments = (dst_numerator.t() / torch.sum(dst_numerator, 1)).t()       #soft assignment using t-distribution
    
    clusters_similar = torch.cosine_similarity(src_soft_assignments, dst_soft_assignments)
    
#     nodes_distance = torch.norm(lhs[:, None]-rhs, dim=2, p=2)
#     nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
    
    distance_U_V = F.pairwise_distance(lhs, rhs)
    nodes_distance = torch.div(distance_U_V, torch.max(distance_U_V))
#     distance_U_V = 1.0 / (1.0 + (distance_U_V / DEC))
#     nodes_distance = distance_U_V**power
    
    distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
    LINK_PROB = torch.exp(-distance_similar)

#     LINK_PROB = 0.5*nodes_distance + 0.5*clusters_similar

    return LINK_PROB

In [4]:
"""  Given the cluster centroids, update the node representation  """
class get_node_emb(nn.Module):
    def __init__(self, node_emb, cluster_emb, alpha=1.0):
        super(get_node_emb, self).__init__()
        self.alpha = alpha
        self.power = float(self.alpha + 1) / 2 
        self.nodes_embedding = Parameter(node_emb)
        self.anchor_emb = cluster_emb
        
    def forward(self, idx_U, idx_V):
        lhs = self.nodes_embedding[idx_U]
        rhs = self.nodes_embedding[idx_V]
        
        # Calculate the attention score (in nodes) for each src node to each cluster
        src_norm_squared = torch.sum((lhs.unsqueeze(1) - self.anchor_emb)**2, 2)   # Distance from the node to the center of mass
        src_numerator = 1.0 / (1.0 + (src_norm_squared / DEC))
        src_numerator = src_numerator**self.power   
        src_soft_assignments = (src_numerator.t() / torch.sum(src_numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the attention score (in nodes) for each dst node to each cluster
        dst_norm_squared = torch.sum((rhs.unsqueeze(1) - self.anchor_emb)**2, 2)   # Distance from the node to the center of mass
        dst_numerator = 1.0 / (1.0 + (dst_norm_squared / DEC))
        dst_numerator = dst_numerator**self.power   
        dst_soft_assignments = (dst_numerator.t() / torch.sum(dst_numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
#         prod = torch.mm(src_soft_assignments, dst_soft_assignments.t())       #  numerator
#         norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
#         clusters_similar = prod.div(torch.mm(norm.t(),norm))
        clusters_similar = torch.cosine_similarity(src_soft_assignments, dst_soft_assignments)
        
        # Calculate the distance between nodes: smaller values favor edge formation
#         nodes_distance = torch.norm(lhs[:, None]-rhs, dim=2, p=2)
#         nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # distance_U_V = torch.sum((lhs.unsqueeze(1) - rhs)**2, 2)
        distance_U_V = F.pairwise_distance(lhs, rhs)
        nodes_distance = torch.div(distance_U_V, torch.max(distance_U_V))
#         distance_U_V = 1.0 / (1.0 + (distance_U_V / DEC))
#         nodes_distance = distance_U_V**self.power
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        LINK_PROB = torch.exp(-distance_similar)

#         LINK_PROB = 0.5*nodes_distance + 0.5*clusters_similar
        
        return LINK_PROB


class update_nodes_embedding(nn.Module):
    def __init__(self, NODE_EMB, CLUSTERS_EMB):
        super(update_nodes_embedding, self).__init__()
        self.prediction_module = get_node_emb(NODE_EMB, CLUSTERS_EMB)     
        self.optimizer = torch.optim.SGD(params=self.prediction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, DATA_TRAIN, DATA_TEST):
        self.prediction_module.train()
        for epoch in range(5):
#             self.optimizer.zero_grad()
#             graph_reconstruction = self.reconstruction_module(CC)
#             graph_train = torch.take(graph_reconstruction, edge_train)
#             loss = self.loss_function(g, graph_train)
#             loss.backward()
#             self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
            total_loss = 0

            ##################### 批训练 #################
            for step, (edge_U, edge_V, edge_label) in enumerate(DATA_TRAIN):
            
                nodes_similar = self.prediction_module(edge_U, edge_V)

                loss = self.loss_function(nodes_similar, edge_label)

                self.optimizer.zero_grad()
            
                loss.backward()

                self.optimizer.step()

                total_loss += loss.item()
            
            print(f'Epoch: {epoch:02d}, Loss: {total_loss:.4f}')
    
        FINAL_NODE_EMDS = self.prediction_module.nodes_embedding.detach()
        FINAL_cluster_EMDS = self.prediction_module.anchor_emb.detach()
    
        all_predict_edges = []
        all_true_edges = []
        for _, (test_idx_U, test_idx_V, test_edge_label) in enumerate(DATA_TEST):
        
            test_lhs = FINAL_NODE_EMDS[test_idx_U]
            test_rhs = FINAL_NODE_EMDS[test_idx_V]

            predict_test_edges = predict_edges(test_lhs, test_rhs, FINAL_cluster_EMDS)

            predict_label = np.array(predict_test_edges)
            true_label = np.array(test_edge_label)
            all_predict_edges = np.concatenate((all_predict_edges, predict_label), axis=0)
            all_true_edges = np.concatenate((all_true_edges, true_label),axis=0)
    
        predict_ap, predict_auc = computer_prediction(all_true_edges, all_predict_edges)
        print('epoch: ', epoch, 'predict_auc_pr = ', predict_ap, 'predict_auc_roc = ', predict_auc)

        return FINAL_NODE_EMDS, predict_ap, predict_auc

In [18]:
"""  Type-centroid update aimed at maximizing the prediction effect   """
class get_clusters_emb(nn.Module):
    """  Given the node representation, update the cluster centroids  """
    def __init__(self, node_emb, cluster_emb, alpha=1.0):
        super(get_clusters_emb, self).__init__()
        self.alpha = alpha
        self.power = float(self.alpha + 1) / 2
        self.anchor_emb = Parameter(cluster_emb)
        self.nodes_embedding = node_emb
        
    
    def forward(self, idx_U, idx_V):
        lhs = self.nodes_embedding[idx_U]
        rhs = self.nodes_embedding[idx_V]
        
        # Calculate the attention score (in nodes) for each src node to each cluster
        src_norm_squared = torch.sum((lhs.unsqueeze(1) - self.anchor_emb)**2, 2)   # Distance from the node to the center of mass
        src_numerator = 1.0 / (1.0 + (src_norm_squared / DEC))
        src_numerator = src_numerator**self.power   
        src_soft_assignments = (src_numerator.t() / torch.sum(src_numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the attention score (in nodes) for each dst node to each cluster
        dst_norm_squared = torch.sum((rhs.unsqueeze(1) - self.anchor_emb)**2, 2)   # Distance from the node to the center of mass
        dst_numerator = 1.0 / (1.0 + (dst_norm_squared / DEC))
        dst_numerator = dst_numerator**self.power   
        dst_soft_assignments = (dst_numerator.t() / torch.sum(dst_numerator, 1)).t()       #soft assignment using t-distribution
        
        # Calculate the cluster similarity between nodes (cosine similarity): the larger the value, the more conducive to edge formation
#         prod = torch.mm(src_soft_assignments, dst_soft_assignments.t())       #  numerator
#         norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)      #    denominator 
#         clusters_similar = prod.div(torch.mm(norm.t(),norm))
        clusters_similar = torch.cosine_similarity(src_soft_assignments, dst_soft_assignments)
        
        # Calculate the distance between nodes: smaller values favor edge formation
#         nodes_distance = torch.norm(lhs[:, None]-rhs, dim=2, p=2)
#         nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # distance_U_V = torch.sum((lhs.unsqueeze(1) - rhs)**2, 2)
        distance_U_V = F.pairwise_distance(lhs, rhs)
        nodes_distance = torch.div(distance_U_V, torch.max(distance_U_V))
#         distance_U_V = 1.0 / (1.0 + (distance_U_V / DEC))
#         nodes_distance = distance_U_V**self.power
        
        # The formation probability of the edges is calculated
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        LINK_PROB = torch.exp(-distance_similar)
#         LINK_PROB = 0.5*nodes_distance + 0.5*clusters_similar
        
        
        return LINK_PROB
    
    
class update_cluster_embedding(nn.Module):
    def __init__(self, NODE_EMB, CLUSTERS_EMB):
        super(update_cluster_embedding, self).__init__()
        self.prediction_module = get_clusters_emb(NODE_EMB, CLUSTERS_EMB)     
        self.optimizer = torch.optim.SGD(params=self.prediction_module.parameters(), lr=LR, momentum=0.9)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, DATA_TRAIN, DATA_TEST):
        self.prediction_module.train()
        for epoch in range(5):
#             self.optimizer.zero_grad()
#             graph_reconstruction = self.reconstruction_module(CC)
#             graph_train = torch.take(graph_reconstruction, edge_train)
#             loss = self.loss_function(g, graph_train)
#             loss.backward()
#             self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
            total_loss = 0

            ##################### 批训练 #################
            for step, (edge_U, edge_V, edge_label) in enumerate(DATA_TRAIN):
            
                nodes_similar = self.prediction_module(edge_U, edge_V)

                loss = self.loss_function(nodes_similar, edge_label)

                self.optimizer.zero_grad()
            
                loss.backward()

                self.optimizer.step()

                total_loss += loss.item()
            
            print(f'Epoch: {epoch:02d}, Loss: {total_loss:.4f}')
    
        FINAL_NODE_EMDS = self.prediction_module.nodes_embedding.detach()
        FINAL_cluster_EMDS = self.prediction_module.anchor_emb.detach()
    
        all_predict_edges = []
        all_true_edges = []
        for _, (test_idx_U, test_idx_V, test_edge_label) in enumerate(DATA_TEST):
        
            test_lhs = FINAL_NODE_EMDS[test_idx_U]
            test_rhs = FINAL_NODE_EMDS[test_idx_V]

            predict_test_edges = predict_edges(test_lhs, test_rhs, FINAL_cluster_EMDS)

            predict_label = np.array(predict_test_edges)
            true_label = np.array(test_edge_label)
            all_predict_edges = np.concatenate((all_predict_edges, predict_label), axis=0)
            all_true_edges = np.concatenate((all_true_edges, true_label),axis=0)
    
        predict_ap, predict_auc = computer_prediction(all_true_edges, all_predict_edges)
        print('epoch: ', epoch, 'predict_auc_pr = ', predict_ap, 'predict_auc_roc = ', predict_auc)

        return FINAL_cluster_EMDS, predict_ap, predict_auc

In [19]:
def get_train_test_data(filename_adj, nodes_number):

    edge_index = pd.read_csv(filename_adj, header=None, sep=' ')
    edges = torch.from_numpy(edge_index.values).t().contiguous()
    
    #print("edges number ： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1
        graph_np[edges[1,i], edges[0,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # Only the upper-left corner matrix is taken
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # Only the upper-left corner matrix is taken
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi_edges_number = len(choose_positive)
    train_neg_edges_number = len(choose_negative)

    test_posi_edges_number = len(test_positive)
    test_neg_edges_number = len(test_negative)
    
    train_posi = [list(choose_positive[i]) for i in range(train_posi_edges_number)]
    train_nega = [list(choose_negative[i]) for i in range(train_neg_edges_number)]
    train_index = train_posi + train_nega
    train_edge_index = torch.tensor(train_index)
    train_U_TRAIN_INDEX = train_edge_index[:, 0]
    train_V_TRAIN_INDEX = train_edge_index[:, 1]

    train_edge_label = list(np.ones(train_posi_edges_number)) + list(np.zeros(train_neg_edges_number))
    train_edge_label = torch.tensor(train_edge_label)

    # 先转换成torch能识别的dataset
    train_dataset = data.TensorDataset(train_U_TRAIN_INDEX, train_V_TRAIN_INDEX, train_edge_label.float())

    # 把dataset放入DataLoader
    train_data_loader = data.DataLoader(
        dataset = train_dataset,
        batch_size = 128,             # 每批提取的数量
        shuffle = True,             # 要不要打乱数据（打乱比较好）
        num_workers = 2             # 多少线程来读取数据
    )


    test_posi = [list(test_positive[i]) for i in range(test_posi_edges_number)]
    test_nega = [list(test_negative[i]) for i in range(test_neg_edges_number)]
    test_index = test_posi + test_nega
    test_edge_index = torch.tensor(test_index)
    test_U_TRAIN_INDEX = test_edge_index[:, 0]
    test_V_TRAIN_INDEX = test_edge_index[:, 1]
    test_edge_label = list(np.ones(test_posi_edges_number)) + list(np.zeros(test_neg_edges_number))
    test_edge_label = torch.tensor(test_edge_label)

    # 先转换成torch能识别的dataset
    test_dataset = data.TensorDataset(test_U_TRAIN_INDEX, test_V_TRAIN_INDEX, test_edge_label.float())

    # 把dataset放入DataLoader
    test_data_loader = data.DataLoader(
        dataset = test_dataset,
        batch_size = 128,             # 每批提取的数量
        shuffle = True,             # 要不要打乱数据（打乱比较好）
        num_workers = 1             # 多少线程来读取数据
    )
    
    return train_data_loader, test_data_loader

In [20]:
def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [21]:
class get_ini_emds(nn.Module):
    def __init__(self, dim, nodes_number, num_CLUSTER):
        super(get_ini_emds, self).__init__()
        self.number_clusters = num_CLUSTER
        self.NODES_EMB = nn.Embedding(nodes_number, dim)
        
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_normal_(self.NODES_EMB.weight)

    def forward(self):
        INI_NODES_EMB = self.NODES_EMB.weight.detach()

        NODES_kmeans = KMeans(n_clusters = self.number_clusters, random_state=42).fit(INI_NODES_EMB)
        cluster_centers = NODES_kmeans.cluster_centers_
        anchor_emb = torch.tensor(cluster_centers, dtype=torch.float)     # Initial cluster centroids

        return INI_NODES_EMB, anchor_emb

In [24]:
filename = 'datasets/Celegans.txt'    # The node index starts at 0, sep=None
nodes_number = 297

embedding_dim = 8
n_clusters = 24
alpha = 1
beta = 4.4
DEC = 5.0
LR = 0.4

all_ap_first_ap = []
all_ap_first_auc = []

all_auc_first_ap = []
all_auc_first_auc = []


train_data_loader, test_data_loader = get_train_test_data(filename, nodes_number)

nodes_emds = get_ini_emds(embedding_dim, nodes_number, n_clusters)
raw_nodes_embedding, raw_cluster_centers = nodes_emds()
    
ap_first__ap = 0
ap_first_auc = 0
auc_first__ap = 0
auc_first_auc = 0
for module_epoch in range(20):
    print("######################### Module cycle ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding, raw_cluster_centers)
    raw_nodes_embedding, nodes_ap, nodes_auc = update_nodes_module(train_data_loader, test_data_loader)
    if nodes_ap > ap_first__ap:
        ap_first__ap = nodes_ap
        ap_first_auc = nodes_auc
    if nodes_auc > auc_first_auc:
        auc_first_auc = nodes_auc
        auc_first__ap = nodes_ap
        
    update_CLUSTERS_module = update_cluster_embedding(raw_nodes_embedding, raw_cluster_centers)
    raw_cluster_centers, cluster_ap, cluster_auc = update_CLUSTERS_module(train_data_loader, test_data_loader)
    if cluster_ap > ap_first__ap:
        ap_first__ap = cluster_ap
        ap_first_auc = cluster_auc
    if cluster_auc > auc_first_auc:
        auc_first_auc = cluster_auc
        auc_first__ap = cluster_ap
all_ap_first_ap.append(ap_first__ap)
all_ap_first_auc.append(ap_first_auc)
all_auc_first_ap.append(auc_first__ap)
all_auc_first_auc.append(auc_first_auc)

######################### Module cycle ： 0 ##########################
Epoch: 00, Loss: 1780.7840
Epoch: 01, Loss: 1428.1260
Epoch: 02, Loss: 1343.4122
Epoch: 03, Loss: 1298.0459
Epoch: 04, Loss: 1255.4125
AP：  0.7422924208790077
AUC SCORE:  0.7084910762574365
epoch:  4 predict_auc_pr =  0.7422924208790077 predict_auc_roc =  0.7084910762574365
Epoch: 00, Loss: 1202.2656
Epoch: 01, Loss: 1202.5471
Epoch: 02, Loss: 1204.5871
Epoch: 03, Loss: 1196.3201
Epoch: 04, Loss: 1192.6495
AP：  0.7582220088870051
AUC SCORE:  0.7196322336398053
epoch:  4 predict_auc_pr =  0.7582220088870051 predict_auc_roc =  0.7196322336398053
######################### Module cycle ： 1 ##########################
Epoch: 00, Loss: 1218.5131
Epoch: 01, Loss: 1175.8962
Epoch: 02, Loss: 1145.5180
Epoch: 03, Loss: 1125.2375
Epoch: 04, Loss: 1101.9931
AP：  0.7988658415565674
AUC SCORE:  0.7648025959978366
epoch:  4 predict_auc_pr =  0.7988658415565674 predict_auc_roc =  0.7648025959978366
Epoch: 00, Loss: 1037.0204
Epoch: 0

Epoch: 00, Loss: 718.0312
Epoch: 01, Loss: 715.1085
Epoch: 02, Loss: 717.2638
Epoch: 03, Loss: 710.2417
Epoch: 04, Loss: 718.5746
AP：  0.8550399040504184
AUC SCORE:  0.8785289345592211
epoch:  4 predict_auc_pr =  0.8550399040504184 predict_auc_roc =  0.8785289345592211
######################### Module cycle ： 14 ##########################
Epoch: 00, Loss: 743.4529
Epoch: 01, Loss: 749.3724
Epoch: 02, Loss: 739.9807
Epoch: 03, Loss: 745.4055
Epoch: 04, Loss: 746.3704
AP：  0.8497770278255323
AUC SCORE:  0.8655273120605733
epoch:  4 predict_auc_pr =  0.8497770278255323 predict_auc_roc =  0.8655273120605733
Epoch: 00, Loss: 707.5552
Epoch: 01, Loss: 721.0302
Epoch: 02, Loss: 710.3438
Epoch: 03, Loss: 713.3712
Epoch: 04, Loss: 708.4431
AP：  0.8573503952312405
AUC SCORE:  0.8844997295835587
epoch:  4 predict_auc_pr =  0.8573503952312405 predict_auc_roc =  0.8844997295835587
######################### Module cycle ： 15 ##########################
Epoch: 00, Loss: 748.1851
Epoch: 01, Loss: 747.4